# Information Retrieval Coursework (7071CEM)

Task:
Develop a vertical search engine similar to Google Scholar that only retrieves papers/books published by a member of Coventry University. As such, at least one of the co-authors must be a Coventry University Staff member. 

To this end, the profiles of academic staff at CU available on the CU web site are crawled, and their papers within their profiles are indexed.

## Package Installs

Some packages required for this task are by default not included in the jupyter notebooks package list. A pip-install of these packages is required. 

In [2]:
# Delete the "#" before each subsequent line to install the package

#!pip install scrapy
#!pip install requests
#!pip install BeautifulSoup4
#!pip install nltk
#!pip install gensim
#!pip install xgboost
#!pip install pandastable

## Imports

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import string
import json
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/philip1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/philip1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philip1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/philip1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. Crawler Component

In [4]:
# Define Seed Page
URL = "https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/"

# Define profile URL format. This was obtained by manually examining the profile pages
profile_url = "https://pureportal.coventry.ac.uk/en/persons/"

The seed page, i.e coventry researchers profiles landing page, contains profiles of all academic researchers of the University. As there are up to 2206 results, a limited number are displayed at once (50).

Even if new profiles are added and a new page is required, the crawler will need to crawl through all pages to access every profile. To do this, a function is defined to retrieve the current total number of result pages. 

In [5]:
def get_maximum_page():
    
    first = requests.get(URL)
    soup = BeautifulSoup(first.text, 'html.parser')
    final_page = soup.select('#main-content > div > section > nav > ul > li:nth-child(12) > a')[0]['href']
    fp = final_page.split('=')[-1]
    return int(fp)
    
mx = get_maximum_page()    

Instead of crawling all researchers, this web crawler is designed to specifically find researchers who:
    
1. Have research publications
2. Are part of the "**School of Computing, Electronics and Maths**" 

In [6]:
def check_department(researcher):
    
    l1 = researcher.find('div', class_='rendering_person_short')
      
    for span in l1.find_all('span'):
        # Check department
        if span.text == str('School of Computing, Electronics and Maths'):
            name = researcher.find('h3', class_='title').find('span').text
            return name
        else:
            pass

def create_csv():
    database = pd.DataFrame(columns=['Title', 'Author', 'Published', 'Link'])
    database.to_csv('database.csv')
    
def update_csv(database):
    current_data = pd.read_csv(database, index_col="Unnamed: 0")
    return current_data        

def enter_each_researchers_publication(researcher, url, df):
    
    new_url = url + str(researcher).replace(' ','-').lower() + '/publications/'
    page = requests.get(new_url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="main-content")
    papers = results.find_all("li", class_="list-result-item")
    
    
    for paper in papers:
        title = paper.find('h3', class_='title').find('span')
        author = paper.find('a', class_='link person').find('span')
        date = paper.find('span', class_="date")
        link = paper.find('h3', class_='title').find('a', href=True)['href']
        
        opening = pd.read_csv('database.csv', index_col="Unnamed: 0")
        opening = opening.append({'Title': title.text, 
                                  'Author': author.text, 
                                  'Published': date.text,
                                  'Link': link}, ignore_index=True)
        opening.to_csv('database.csv')
                

In [7]:
## Scrape function
def scrape(mx):
    df = update_csv('database.csv')
    i=0
    while True:
    
        if i > 17:
            break
            
        if i>0:
            url = URL + '?page=' + str(i)
        else:
            url = URL
    
        i = i+1
        # scraping starts here
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="main-content")
        researchers = results.find_all("li", class_="grid-result-item")

        for researcher in researchers:
            # Check if researcher has any papers
            check = researcher.find('div', class_='stacked-trend-widget')
            if check:
                name = check_department(researcher)
                if name is None:
                    pass
                else:
                    enter_each_researchers_publication(name, profile_url, df)

In [8]:
create_csv()

%time scrape(mx) 

CPU times: user 8.45 s, sys: 561 ms, total: 9.02 s
Wall time: 1min 31s


In [9]:
sample_db = pd.read_csv('database.csv').rename(columns={'Unnamed: 0':'SN'})
sample_db
print(f'{sample_db.shape[0]} records were scraped')

396 records were scraped


# 2. Indexing Component

In [10]:
scraped_db = pd.read_csv('database.csv').rename(columns={'Unnamed: 0':'SN'}).reset_index(drop=True)
scraped_db.head()

,SN,Title,Author,Published,Link
0,0,Cross-Layer Multipath Multichannel MAC protoco...,"Abdelshafy, M.",Jun 2018,https://pureportal.coventry.ac.uk/en/publicati...
1,1,Performance evaluation of Receiver Directed Tr...,"Abdelshafy, M.",4 May 2017,https://pureportal.coventry.ac.uk/en/publicati...
2,2,Reliable Multipath Multi-channel Route Migrati...,"Abdelshafy, M.",23 Nov 2017,https://pureportal.coventry.ac.uk/en/publicati...
3,3,Resisting blackhole attacks on MANETs,"Abdelshafy, M.",Jan 2016,https://pureportal.coventry.ac.uk/en/publicati...
4,4,Dynamic Source Routing under Attacks,"Abdelshafy, M.",12 Oct 2015,https://pureportal.coventry.ac.uk/en/publicati...


In [58]:
sample_db.head(7)
#ids = scraped_db["Title"]
#scraped_db[ids.isin(ids[ids.duplicated()])]

,SN,Title,Author,Published,Link
0,0,Cross-Layer Multipath Multichannel MAC protoco...,"Abdelshafy, M.",Jun 2018,https://pureportal.coventry.ac.uk/en/publicati...
1,1,Performance evaluation of Receiver Directed Tr...,"Abdelshafy, M.",4 May 2017,https://pureportal.coventry.ac.uk/en/publicati...
2,2,Reliable Multipath Multi-channel Route Migrati...,"Abdelshafy, M.",23 Nov 2017,https://pureportal.coventry.ac.uk/en/publicati...
3,3,Resisting blackhole attacks on MANETs,"Abdelshafy, M.",Jan 2016,https://pureportal.coventry.ac.uk/en/publicati...
4,4,Dynamic Source Routing under Attacks,"Abdelshafy, M.",12 Oct 2015,https://pureportal.coventry.ac.uk/en/publicati...
5,5,AODV & SAODV under Attack: Performance Comparison,"Abdelshafy, M.",Jun 2014,https://pureportal.coventry.ac.uk/en/publicati...
6,6,AODV Routing Protocol Performance Analysis und...,"Abdelshafy, M.",Jun 2014,https://pureportal.coventry.ac.uk/en/publicati...


In [11]:
single_row = scraped_db.loc[1,:].copy()
single_row

SN                                                           1
Title        Performance evaluation of Receiver Directed Tr...
Author                                          Abdelshafy, M.
Published                                           4 May 2017
Link         https://pureportal.coventry.ac.uk/en/publicati...
Name: 1, dtype: object

## 2.1 Preprocess Text

In [12]:
# Remove stop words
sw = stopwords.words("english")
lemmatizer = WordNetLemmatizer()




def tp1(txt):
    txt = txt.lower()   # Make lowercase
    txt = txt.translate(str.maketrans('',
                                      '',
                                      string.punctuation))   # Remove punctuation marks
    txt = lematize(txt)
    return txt


def fwpt(word):
    tag = pos_tag([word])[0][1][0].upper()
    hash_tag = {"V": wordnet.VERB, "R": wordnet.ADV,"N": wordnet.NOUN,"J": wordnet.ADJ}         
    return hash_tag.get(tag, wordnet.NOUN)

def lematize(text):
        tkns = nltk.word_tokenize(text)
        ax = ""
        for each in tkns:
            if each not in sw:
                ax += lemmatizer.lemmatize(each, fwpt(each)) + " "
        return ax


In [13]:
# Sample title
single_row['Title']

'Performance evaluation of Receiver Directed Transmission protocol with a single transceiver in MANETs'

In [14]:
# Demonstration of lowercase and punctuation removal
tp1(single_row['Title'])

'performance evaluation receiver direct transmission protocol single transceiver manet '

In [15]:
# Demonstration of lematization

lematize(tp1(single_row['Title']))
#lematize(single_row['Title'])

'performance evaluation receiver direct transmission protocol single transceiver manet '

#### Unprocessed

In [59]:
sample_db['Title'].iloc[5]

'AODV & SAODV under Attack: Performance Comparison'

#### Processed

In [60]:
scraped_db['Title'].iloc[5]

'aodv saodv attack performance comparison '

### 2.1.1 Preprocess entire dataframe

In [16]:
processed_db = scraped_db.copy()

def preprocess_df(df):
    df.Title = df.Title.apply(tp1)
    df.Author = df.Author.str.lower()
    df = df.drop(columns=['Author','Published'], axis=1)
    return df
    
preprocess_df(processed_db)
processed_db.head()

,SN,Title,Author,Published,Link
0,0,crosslayer multipath multichannel mac protocol...,"abdelshafy, m.",Jun 2018,https://pureportal.coventry.ac.uk/en/publicati...
1,1,performance evaluation receiver direct transmi...,"abdelshafy, m.",4 May 2017,https://pureportal.coventry.ac.uk/en/publicati...
2,2,reliable multipath multichannel route migratio...,"abdelshafy, m.",23 Nov 2017,https://pureportal.coventry.ac.uk/en/publicati...
3,3,resist blackhole attack manet,"abdelshafy, m.",Jan 2016,https://pureportal.coventry.ac.uk/en/publicati...
4,4,dynamic source rout attack,"abdelshafy, m.",12 Oct 2015,https://pureportal.coventry.ac.uk/en/publicati...


## 2.2 Index Construction

In [17]:
single = processed_db.loc[0,:].copy()
print(single)
indexing_trial = {}

words = single.Title.split()
SN = single.SN
word = words[0]
example = {word: [SN]}

print('=====================================================================')
print('Sample index')
print(example)

SN                                                           0
Title        crosslayer multipath multichannel mac protocol...
Author                                          abdelshafy, m.
Published                                             Jun 2018
Link         https://pureportal.coventry.ac.uk/en/publicati...
Name: 0, dtype: object
Sample index
{'crosslayer': [0]}


In [18]:
## Indexer Function
def apply_index(inputs, index):
    words = inputs.Title.split()
    SN = int(inputs.SN)
    for word in words:
        if word in index.keys():
            if SN not in index[word]:
                index[word].append(SN)
        else:
            index[word] = [SN]
    return index

indx = apply_index(inputs=single, index= {})
#print(indx)

In [19]:
def full_index(df, index):
    for x in range(len(df)):
        inpt = df.loc[x,:]
        ind = apply_index(inputs=inpt, index=index)
    return ind

def construct_index(df, index):
    queue = preprocess_df(df)
    ind = full_index(df=queue, index=index)
    return ind

indexed = full_index(processed_db, 
                     index = {})


indexes = construct_index(df=scraped_db, 
                          index = {})

In [20]:
with open('indexes.json', 'w') as new_f:
    json.dump(indexes, new_f, sort_keys=True, indent=4)
    
with open('indexes.json', 'r') as file:
    data = json.load(file)

def index_2(df, x_path):
    if len(df) > 0:
        with open(x_path, 'r') as file:
            prior_index = json.load(file)
        new_index = construct_index(df = df, index = prior_index)
        with open(x_path, 'w') as new_f:
            json.dump(new_index, new_f, sort_keys=True, indent=4)

In [61]:
len(data)

1258

In [82]:
data

{'10': [111, 377],
 '15360bit': [108],
 '2007': [176],
 '2009': [371],
 '2015': [323],
 '30': [224, 225],
 '3dterrains': [96],
 '40': [252],
 '4th': [371],
 '81': [111, 377],
 'abcsg': [286],
 'ability': [248],
 'absorption': [106],
 'acceleration': [29],
 'access': [66],
 'accommodation': [178, 190],
 'accord': [319],
 'accurate': [129],
 'achievementbased': [220],
 'action': [242, 368],
 'active': [10, 318],
 'activity': [104, 105, 119, 121, 198, 235, 257, 334],
 'activityled': [120, 367],
 'actuator': [98, 101],
 'ad': [2],
 'adapt': [319, 328, 332],
 'adaptation': [52, 56],
 'adaptive': [55, 102, 182, 207, 214, 215, 268, 312, 324],
 'adaptivity': [53, 54, 57, 58],
 'adolescent': [185, 189, 192, 200, 221],
 'adult': [140, 312],
 'advance': [170],
 'adversarial': [124, 135],
 'aerospace': [307, 310],
 'affect': [317, 320, 323],
 'affectaware': [313, 314, 317],
 'affective': [314, 315, 316, 319, 324, 330, 331, 332],
 'affine': [95],
 'agent': [77, 174, 175, 176, 236, 237, 335, 370],
 

## 3.  Query Processor

In [65]:
def demonstrate_query_processing():
    sample = input('Enter Search Terms: ')
    processed_query = tp1(sample)
    #print(f'User Search Query: {sample}')
    print(f'Processed Search Query: {processed_query}')
    return processed_query
    
#demonstrate_query_processing()

### 3.1.  Split Query into individual terms

In [66]:
def split_query(terms):
    each = tp1(terms)
    return each.split()

dqp = demonstrate_query_processing()
dqp
print(f'Split Query: {split_query(dqp)}')

Enter Search Terms: This is a demonstration of query processing functionality
Processed Search Query: demonstration query processing functionality 
Split Query: ['demonstration', 'query', 'processing', 'functionality']


### 3.2.  Boolean Functionalities

In [24]:
def union(lists):
    union = list(set.union(*map(set, lists)))
    union.sort()
    return union

def intersection(lists):
    intersect = list(set.intersection(*map(set, lists)))
    intersect.sort()
    return intersect

### 3.3. Search Engine Function

In [25]:
def vertical_search_engine(df, query, index=indexes):
    query_split = split_query(query)
    retrieved = []
    for word in query_split:
        if word in index.keys():
            retrieved.append(index[word])
            
            
    # Ranked Retrieval
    if len(retrieved)>0:
        high_rank_result = intersection(retrieved)
        low_rank_result = union(retrieved) 
        c = [x for x in low_rank_result if x not in high_rank_result]      
        high_rank_result.extend(c)
        result = high_rank_result
        
        final_output = df[df.SN.isin(result)].reset_index(drop=True)
    
        # Return result in order of Intersection ----> Union
        dummy = pd.Series(result, name = 'SN').to_frame()
        result = pd.merge(dummy, final_output, on='SN', how = 'left')
        
    else:
        result = 'No result found'
    
    return result

In [26]:
def test_search_engine():
    xtest = scraped_db.copy()
    query = input("Enter your search query: ")
    return vertical_search_engine(xtest, query, indexed)
    
test_search_engine()

Enter your search query: databases


,SN,Title,Author,Published,Link
0,81,hisql frontend query system historical database,"anane, r.",Nov 1997,https://pureportal.coventry.ac.uk/en/publicati...
1,295,speedingup similarity search time series datab...,"muhammad fuad, m. m.",11 Nov 2010,https://pureportal.coventry.ac.uk/en/publicati...
2,343,graphical data display database query help use...,"grawemeyer, b.",1 Jan 2005,https://pureportal.coventry.ac.uk/en/publicati...
3,344,effect knowledgeofexternalrepresentations upon...,"grawemeyer, b.",1 Jan 2004,https://pureportal.coventry.ac.uk/en/publicati...


In [71]:
def final_engine(results):
    if type(results) != 'list':
        return results
        #print(results)
    else:
        for i in range(len(results)):
            printout = results.loc[i, :]
            #print(printout['Title'])
            #print(printout['Author'])
            #print(printout['Published'])
            #print(printout['Link'])
            #print('')

In [74]:
scraped_db['Title'].iloc[24]

'design visible light communication system deep sea diver base analogue frequency modulation '

In [73]:
final_engine(test_search_engine())

Enter your search query: Data Base Machine


,SN,Title,Author,Published,Link
0,247,cps data stream analytics base machine learn c...,"fei, x.",Jan 2019,https://pureportal.coventry.ac.uk/en/publicati...
1,258,energy consumption data base machine anomaly d...,"fei, x.",Nov 2014,https://pureportal.coventry.ac.uk/en/publicati...
2,24,design visible light communication system deep...,"ahmad, z.",22 Sep 2016,https://pureportal.coventry.ac.uk/en/publicati...
3,42,digital data processing stilbene,"amiri, m.",30 Jan 2014,https://pureportal.coventry.ac.uk/en/publicati...
4,43,digital twoparametric processing output data r...,"amiri, m.",Apr 2014,https://pureportal.coventry.ac.uk/en/publicati...
5,48,discrimination neutron photon signal use time ...,"amiri, m.",Apr 2012,https://pureportal.coventry.ac.uk/en/publicati...
6,55,design usability evaluation adaptive elearning...,"anane, r.",Aug 2015,https://pureportal.coventry.ac.uk/en/publicati...
7,73,stateless data concealment distribute system,"anane, r.",Mar 2008,https://pureportal.coventry.ac.uk/en/publicati...
8,80,data mining serial document,"anane, r.",Aug 2001,https://pureportal.coventry.ac.uk/en/publicati...
9,85,fem approach diagnosis induction machine nonad...,"arvanitakis, i.",17 Jun 2019,https://pureportal.coventry.ac.uk/en/publicati...


## 4. Schedule Crawler for every week

To demonstrate a weekly scheduled crawling, the following parameters are defined:

* `interval` : Represents number of days in reality. In this code, it represents only seconds for demonstration


In [29]:
days = 0
interval = 7
while days <= 1:
    scrape(mx)
    print(f"Crawled at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f'Next crawl scheduled after {interval} days')
    time.sleep(interval)
    days = days + 1

Crawled at 2021-08-01 02:02:51
Next crawl scheduled after 7 days
Crawled at 2021-08-01 02:05:50
Next crawl scheduled after 7 days


# 5. GUI

In [40]:
from tkinter import *
from PIL import Image, ImageTk 
from tkinter import scrolledtext
from pandastable import Table, TableModel
from contextlib import suppress
import warnings
warnings.filterwarnings('ignore')


image1 = Image.open('coventry-university-logo.png')
resized_image1 = image1.resize((500,300), Image.ANTIALIAS)

def new_gui(pic):
    window = Tk()
    window.title("Coventry University Oracle")
    window.geometry('1100x600')
    
    lbl = Label(window, text="CEM Search Engine", font=("Arial Bold", 30), padx=5, pady=5)
    lbl.grid(column=1, row=0)
    
    lbl2 = Label(window, text="Enter your search query here ===>", font=("Arial", 15), padx=5, pady=5)
    lbl2.grid(column=0, row=1)
    
    
    img = ImageTk.PhotoImage(pic)
    
    lbl3 = Label(image=img)
    lbl3.image = img
    lbl3.grid(column=1, row=3, padx=5, pady=5)
    
    
    
    query = Entry(window,width=40)
    query.grid(column=1, row=1,  padx=5, pady=5)
    
    results = Canvas(window, height=30, width=250)
    results.grid(column=1, row=2, padx=5, pady=5)
    
    # Entry
    def getInputBoxValue():
        userInput = query.get()
        return userInput

    
    # Button
    def clicked():
        search()
        #pass
        
    def no_result():
        messagebox.showwarning("Warning", "No results found. Please try different search terms")
        
    
    def search():
        xtest = scraped_db.copy()
        q = query.get()
        f = Frame(window)
        df = vertical_search_engine(xtest, q, indexed)
        if type(df) == str:
            no_result()
        else:
            pt = Table(results)
            try:
                table = pt = Table(results, dataframe=df)
                pt.show()
            except AttributeError:
                pass
            
    def close_window():
        if messagebox.askokcancel("Quit", "Quit Programme?"):
            window.destroy()
        
    
    btn = Button(window, text="Search", command=clicked)
    btn.grid(column=2, row=1)
    
    

    window.protocol("WM_DELETE_WINDOW", close_window)       
    window.mainloop()

In [83]:
new_gui(image1)